In [ ]:
import re

class StringCalculator:
    def add(self, numbers: str) -> int:
        if not numbers:
            return 0

        # Normalize the string
        numbers = numbers.strip().replace("\\n", "\n")
        default_delimiters = [',', '\n']  # Default delimiters
        custom_delimiter = None
        is_custom_delimiter = False

        # Check for custom delimiters
        if numbers.startswith("//"):
            delimiter_end = numbers.find('\n')
            if delimiter_end == -1:
                raise ValueError("Custom delimiter format must include '\\n'")

            # Extract custom delimiter
            custom_delimiter = numbers[2:delimiter_end]

            # Validate custom delimiter format
            if custom_delimiter.startswith('[') and custom_delimiter.endswith(']'):
                custom_delimiter = custom_delimiter[1:-1]  # Remove brackets
            elif len(custom_delimiter) != 1:
                raise ValueError("Custom delimiter must be a single character or enclosed in brackets")

            # Strip the custom delimiter line
            numbers = numbers[delimiter_end + 1:]
            is_custom_delimiter = True

        # Build the delimiter pattern
        delimiters = [custom_delimiter] if is_custom_delimiter else default_delimiters
        pattern = '|'.join(map(re.escape, delimiters))

        # Validate and split numbers
        tokens = re.split(pattern, numbers)
        nums = []
        negatives = []

        for token in tokens:
            if token.strip():  # Skip empty tokens
                try:
                    n = int(token)
                    if n < 0:
                        negatives.append(n)
                    nums.append(n)
                except ValueError:
                    raise ValueError(f"Invalid number: '{token}'")

        # Strict validation for custom delimiter use
        if is_custom_delimiter:
            for i, char in enumerate(numbers):
                if char in default_delimiters and char != custom_delimiter:
                    raise ValueError(
                        f"Invalid input: Default delimiters '{char}' not allowed with custom delimiter '{custom_delimiter}'"
                    )

        # Handle negatives
        if negatives:
            raise ValueError(f"Negative numbers not allowed: {', '.join(map(str, negatives))}")

        return sum(nums)

def run_tests():
    calc = StringCalculator()
    test_cases = [
        ("", 0, "Empty string"),  # Test case for empty string
        ("//,\n2,3,4", 9, "Custom delimiter ',' matching default delimiter"),
        ("//;\n1;2;3", 6, "Custom delimiter ';'"),
        ("//{\n2{3{4", 9, "Custom delimiter '{'"),
        ("//?\n2?3?4", 9, "Custom delimiter '?'"),
        ("1\n2,3", 6, "Default delimiters"),
        ("54\n9,7\n1", 71, "Default delimiters with mixed newlines"),
    ]

    for input_str, expected, description in test_cases:
        try:
            result = calc.add(input_str)
            assert result == expected, f"{description}: Expected {expected}, got {result}"
            print(f"✓ {description}")
        except AssertionError as e:
            print(f"✗ {str(e)}")
        except ValueError as e:
            print(f"✗ {description}: {str(e)}")

    negative_tests = [
        ("//?\n2>3", "Invalid input: Default delimiters '>' not allowed with custom delimiter '?'"),
        ("//;\n1;2,3", "Invalid input: Default delimiters ',' not allowed with custom delimiter ';'"),
        ("//[\n4,5", "Custom delimiter format must include '\\n'"),
    ]

    for input_str, expected_error in negative_tests:
        try:
            calc.add(input_str)
            print(f"✗ Failed to raise error for {input_str}")
        except ValueError as e:
            if str(e) == expected_error:
                print(f"✓ Correct error for {input_str}")
            else:
                print(f"✗ Wrong error message for {input_str}")
                print(f"Expected: {expected_error}")
                print(f"Got: {str(e)}")

def user_interface():
    calc = StringCalculator()
    print("\n=== String Calculator ===")
    print("Type 'exit' to quit\n")

    while True:
        try:
            numbers = input("Enter numbers: ")
            if numbers.lower() == 'exit':
                break
            result = calc.add(numbers)
            print(f"Result: {result}")
        except ValueError as e:
            print(f"Error: {str(e)}")
        print()

if __name__ == "__main__":
    run_tests()
    user_interface()


✓ Empty string
✓ Custom delimiter ',' matching default delimiter
✓ Custom delimiter ';'
✓ Custom delimiter '{'
✓ Custom delimiter '?'
✓ Default delimiters
✓ Default delimiters with mixed newlines
✗ Wrong error message for //?
2>3
Expected: Invalid input: Default delimiters '>' not allowed with custom delimiter '?'
Got: Invalid number: '2>3'
✗ Wrong error message for //;
1;2,3
Expected: Invalid input: Default delimiters ',' not allowed with custom delimiter ';'
Got: Invalid number: '2,3'
✗ Wrong error message for //[
4,5
Expected: Custom delimiter format must include '\n'
Got: Invalid number: '4,5'

=== String Calculator ===
Type 'exit' to quit

